<a href="https://colab.research.google.com/github/matthewpecsok/data_engineering/blob/main/tutorials/de_tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Jupyter!

This is a Jupyter notebook. Some blocks are simply text and others are Python code or shell commands.

We run the notebook from Top->Bottom in sequence with each block of code often requiring the prior block of code to have been executed.

# Step 1: Install required packages

The standard Python distribution doesn't contain all the functionality required for many tasks. To extend the functionality we install packages. Packages can be installed by using pip (package installer for Python) which is a Python script. To call this script we use !

### !

In order to call pip we can use the ! prefix to tell the Jupyer notebook to execute the code as a shell command. Shell commands are a common way to interact with the operating system on unix/linux platforms. Your Google Colab environment is running the Ubuntu Linux operating system.

## pip details

### pip -q

The "-q" flag tells the pip to be quiet about its output. This hides more verbose output that would normally be printed when the packages install. We don't want to see all that output so we use the "-q" option to tell pip to hide it.

### install --upgrade

The install command tells pip we are planning to install a package. Since the ipython package is already installed we use the --upgrade flag to tell pip to upgrade ipython to the latest version.

### Which package?

Finally we tell pip which package we want to install. In this case we are installing/upgrading 2 packages, ipython and ipython-sql.

In [19]:
!pip -q install --upgrade ipython
!pip -q install --upgrade ipython-sql

# Step 2: Import sqlite3 package

sqlite3 is a package that allows us to use a lightweight, file-based SQL database. We'll use this simple database to perform common database operations without needing to install a more complex database like MySql or Postres. For our purposes a simple flat file database is sufficient.

In [20]:
import sqlite3

use the shell command 'wget' to retrieve the movies database file from github. You can see it in your files path on the left nav bar in colab after running the code.

If you attempt to run this code on a windows operating system it will fail because wget isn't an available command on windows, only unix based platforms.

# Step 3: Download the database to a local file

In [21]:
!wget -O movies.db https://github.com/matthewpecsok/data_engineering/blob/main/data/movies.sqlite?raw=true

--2024-07-29 17:50:27--  https://github.com/matthewpecsok/data_engineering/blob/main/data/movies.sqlite?raw=true
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/matthewpecsok/data_engineering/raw/main/data/movies.sqlite [following]
--2024-07-29 17:50:28--  https://github.com/matthewpecsok/data_engineering/raw/main/data/movies.sqlite
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/movies.sqlite [following]
--2024-07-29 17:50:28--  https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/movies.sqlite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|18

# Step 4: Create a sqlite connection object.

To interact with the database we can create a connection object. We'll call it 'movie_con' but be aware we can give it any name we like, and we can have multiple connection objects to multiple databases open at any time.

In [22]:
movie_con = sqlite3.connect('movies.db')

In [37]:
type(movie_con) # use type to tell us what type of object movie_con is.

sqlite3.Connection

In [23]:
# here we setup Jupyter to allow us to interact with the sqlite database easily within code blocks.

%load_ext sql
%sql sqlite:///movies.db

%config SqlMagic.autopandas = True # automatically return a pandas dataframe object.
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# What table names are in our database?

Use %% magic to run a multi-line sql statement querying the sqlite_master table which tells us all of the tables in the database.


The two tables of interest are directors and movies.

Let's explore those tables.

In [24]:
%%sql SELECT name, sql FROM sqlite_master
WHERE type='table'
ORDER BY name;

,name,sql
0,directors,"CREATE TABLE `directors` (\n`name` TEXT,\n`id`..."
1,movies,CREATE TABLE `movies` (\n`id` INTEGER PRIMARY ...
2,sqlite_sequence,"CREATE TABLE sqlite_sequence(name,seq)"


# sql count

%sql runs a single line sql statement. How many rows are in our movies table?



In [38]:
%sql select count(1) as movie_count from movies

,movie_count
0,4773


How many rows in directors table?

In [39]:
%sql select count(1) as director_count from directors

,director_count
0,2349


Show the first 5 rows in the movies table.

* The id column is the primary key of the table
* original_title is the original name of the movie
* budget is the cost of the movie (in dollars)
* popularity is a score for how popular th emovie was.
* revenue (in dollars, how much the movie brought in)
* title is the name of the movie
* vote_average a score of votes on a 10 point scale
* vote_count: the count of votes
* overview: description of the movie
* tagline: the tagline of the movie
* uid (ignorable)
* director_id: foreign key to the director table.


In [27]:
%sql select * from movies limit 5

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766


# sql limit

Show the first 5 rows in the directors table.

In [28]:
%sql select * from directors limit 5

,name,id,gender,uid,department
0,James Cameron,4762,2,2710,Directing
1,Gore Verbinski,4763,2,1704,Directing
2,Sam Mendes,4764,2,39,Directing
3,Christopher Nolan,4765,2,525,Directing
4,Andrew Stanton,4766,2,7,Directing


# sql max function

Show both the maximum budget and maximum revenue from all movies. Note these may be from 2 different movies.

In [29]:
%sql select max(budget),max(revenue) from movies

,max(budget),max(revenue)
0,380000000,2787965087


Which movie(s) had the max budget?

In [40]:
%sql select * from movies where budget = 380000000

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43614,Pirates of the Caribbean: On Stranger Tides,380000000,135,2011-05-14,1045713802,Pirates of the Caribbean: On Stranger Tides,6.4,4948,Captain Jack Sparrow crosses paths with a woma...,Live Forever Or Die Trying.,1865,4775


Which movie(s) had the max revenue??

In [41]:
%sql select * from movies where revenue = 2787965087

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762


select specific columns from the movie table, sort the dataset by the budget in descending order, limit to the top 10 budgets.

left join to the directors table so we can also retrieve the director's name.

In [51]:
%sql select budget,title,d.name from movies m left join directors d on m.director_id = d.id order by budget desc limit 10

,budget,title,name
0,380000000,Pirates of the Caribbean: On Stranger Tides,Rob Marshall
1,300000000,Pirates of the Caribbean: At World's End,Gore Verbinski
2,280000000,Avengers: Age of Ultron,Joss Whedon
3,270000000,Superman Returns,Bryan Singer
4,260000000,John Carter,Andrew Stanton
5,260000000,Tangled,Byron Howard
6,258000000,Spider-Man 3,Sam Raimi
7,255000000,The Lone Ranger,Gore Verbinski
8,250000000,The Dark Knight Rises,Christopher Nolan
9,250000000,Harry Potter and the Half-Blood Prince,David Yates


We might be curious what the earliest release data was, and the latest release date. Finally we might want to know what range of years are in our dataset.

We can subtract the min release_date from the max_release date to give us the count of years in the dataset.  

In [31]:
%sql select min(release_date),max(release_date),max(release_date) - min(release_date) as years_of_releases from movies

,min(release_date),max(release_date),years_of_releases
0,1916-09-04,2017-02-03,101


# sql avg and group by

We can compute the average budget and revenue over our dataset grouped by director. We then show which directors had the highest average budget.

While Rob Marshal had the most expensive movie by budget his average movie budget was less than other averages. It's worth noting that many directors such as Byron Howard only had 1 movie in the dataset.

In [52]:
%%sql select

avg(budget),
avg(revenue),
name as director_name,
count(1) as movie_count

from movies m left join directors d on m.director_id = d.id
group by director_id order by (avg(budget)) desc
limit 8

,avg(budget),avg(revenue),director_name,movie_count
0,2.600000e+08,5.917949e+08,Byron Howard,1
1,2.000000e+08,1.066970e+09,Lee Unkrich,1
2,2.000000e+08,7.435596e+08,Dan Scanlon,1
3,1.933333e+08,7.429717e+08,David Yates,3
4,1.850000e+08,5.389832e+08,Brenda Chapman,1
5,1.800000e+08,7.585398e+08,Robert Stromberg,1
6,1.796667e+08,9.879437e+08,Joss Whedon,3
7,1.750000e+08,1.509625e+08,Carl Rinsch,1


# sql distinct

In [33]:
%sql select distinct(department) from directors

,department
0,Directing


In [53]:
%%sql select *

from movies m
left join directors d on m.director_id = d.id


limit 5

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id,name,id,gender,uid,department
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762,James Cameron,4762,2,2710,Directing
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763,Gore Verbinski,4763,2,1704,Directing
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764,Sam Mendes,4764,2,39,Directing
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765,Christopher Nolan,4765,2,525,Directing
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766,Andrew Stanton,4766,2,7,Directing


# comparing max and average budgets by director.



## Top 5 directors by average budget.

In [57]:
%%sql select
avg(budget),
max(budget),
avg(revenue),
max(revenue),
director_id,
name,
count(1) as movie_count


from movies m
left join directors d on m.director_id = d.id

group by director_id
order by avg(budget)
desc
limit 5


,avg(budget),max(budget),avg(revenue),max(revenue),director_id,name,movie_count
0,2.600000e+08,260000000,5.917949e+08,591794936,4768,Byron Howard,1
1,2.000000e+08,200000000,1.066970e+09,1066969703,4792,Lee Unkrich,1
2,2.000000e+08,200000000,7.435596e+08,743559607,4787,Dan Scanlon,1
3,1.933333e+08,250000000,7.429717e+08,938212738,4770,David Yates,3
4,1.850000e+08,185000000,5.389832e+08,538983207,4801,Brenda Chapman,1


# sql like statement

Query for any movie with the words star wars. Notice that sqlite is case insensitive.

'% %' searches for any text within the column


In [58]:
%sql select title from movies where title like '%star wars%'

,title
0,Star Wars: Episode III - Revenge of the Sith
1,Star Wars: Episode II - Attack of the Clones
2,Star Wars: Episode I - The Phantom Menace
3,Star Wars
4,Star Wars: Clone Wars: Volume 1


# sql ends with %

search for any title ending with the word 'star'.

In [59]:
%sql select title from movies where title like '%star'

,title
0,Rock Star
1,Dickie Roberts: Former Child Star
2,Superstar
3,Bucky Larson: Born to Be a Star
4,Bright Star
5,My Lucky Star
6,Lone Star


In [61]:
%sql select * from movies where overview like '%comedy%'

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43831,The Croods,135000000,64,2013-03-20,585178928,The Croods,6.8,2399,The Croods is a prehistoric comedy adventure t...,Meet the first modern family.,49519,4825
1,43845,Mr. & Mrs. Smith,110000000,44,2005-06-07,478207520,Mr. & Mrs. Smith,6.5,2965,After five (or six) years of vanilla-wedded bl...,Smart and sexy.,787,4813
2,44033,Grown Ups 2,80000000,45,2013-07-11,246984278,Grown Ups 2,5.8,1155,The all-star comedy cast from Grown Ups return...,Just because they're a little older doesn't me...,109418,4964
3,44057,Zookeeper,80000000,27,2011-07-06,169852759,Zookeeper,5.3,498,A comedy about a zookeeper who might be great ...,Welcome to his jungle.,38317,4917
4,44070,Mars Attacks!,70000000,44,1996-12-12,101371017,Mars Attacks!,6.1,1509,'We come in peace' is not what those green men...,Nice planet. We'll take it!,75,4785
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,48141,The Living Wake,0,0,2007-01-01,0,The Living Wake,5.0,1,“The Living Wake” is a dark comedy set in a ti...,None,43653,6944
87,48229,Lovely & Amazing,250000,1,2001-08-31,4186931,Lovely & Amazing,6.3,23,Self-esteem and insecurity are at the heart of...,None,50035,6204
88,48234,American Desi,250000,0,2001-01-01,0,American Desi,5.0,4,This is a fun-loving romantic comedy reminisce...,None,32222,6998
89,48347,The Dirties,0,0,2013-10-04,0,The Dirties,6.0,42,Two best friends are filming a comedy about ge...,None,159770,7073


# sql greater than, greater than or equal

there are some popular movies... that only 1 person voted for.

In [64]:
%sql select * from movies where vote_average > 9

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,47116,Stiff Upper Lips,0,0,1998-06-12,0,Stiff Upper Lips,10.0,1,Stiff Upper Lips is a broad parody of British ...,None,89861,6332
1,47642,"Dancer, Texas Pop. 81",0,0,1998-05-01,565592,"Dancer, Texas Pop. 81",10.0,1,"Four guys, best friends, have grown up togethe...",in the middle of nowhere they had everything,78373,5463


In [68]:
%sql select * from movies where vote_average >= 8.5 and vote_count >= 10

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,45478,The Shawshank Redemption,25000000,136,1994-09-23,28341469,The Shawshank Redemption,8.5,8205,Framed in the 1940s for the double murder of h...,Fear can hold you prisoner. Hope can set you f...,278,5051


# sql having clause

We can group by and then use the having clause to filter records on aggregated data. For example we can filter movies that have directors who have directed at least 5 movies.

There are 211 directors who have directed at least 5 movies.

In [71]:
%sql select d.name,count(1) from movies m left join directors d on m.director_id = d.id group by director_id having count(1) >= 5 order by count(1) desc

,name,count(1)
0,Steven Spielberg,27
1,Woody Allen,21
2,Clint Eastwood,20
3,Martin Scorsese,20
4,Spike Lee,16
...,...,...
206,Kevin Reynolds,5
207,Justin Lin,5
208,Guillermo del Toro,5
209,Rob Marshall,5
